THIS IS THE NOTEBEOOK WHERE WE ARE GOING TO FIRST INITIALIZE A MODEL AND THEN THEN EMBEDD THE DATA INTO THE RAG PIPELINE


In [1]:
import pandas as pd
from langchain_core.documents import Document

# Load the FIFA matches data
df = pd.read_csv("Fifa_world_cup_matches.csv")  # adjust path if needed

# Quick peek
print(df.head())
print(df.columns)


           team1         team2 possession team1 possession team2  \
0          QATAR       ECUADOR              42%              50%   
1        ENGLAND          IRAN              72%              19%   
2        SENEGAL   NETHERLANDS              44%              45%   
3  UNITED STATES         WALES              51%              39%   
4      ARGENTINA  SAUDI ARABIA              64%              24%   

  possession in contest  number of goals team1  number of goals team2  \
0                    8%                      0                      2   
1                    9%                      6                      2   
2                   11%                      0                      2   
3                   10%                      1                      1   
4                   12%                      1                      2   

          date     hour category  ...  penalties scored team1  \
0  20 NOV 2022  17 : 00  Group A  ...                       0   
1  21 NOV 2022  14 : 0

converting each row of the data into text to embedded into the vector Store

In [2]:
df.columns


Index(['team1', 'team2', 'possession team1', 'possession team2',
       'possession in contest', 'number of goals team1',
       'number of goals team2', 'date', 'hour', 'category',
       'total attempts team1', 'total attempts team2', 'conceded team1',
       'conceded team2', 'goal inside the penalty area team1',
       'goal inside the penalty area team2',
       'goal outside the penalty area team1',
       'goal outside the penalty area team2', 'assists team1', 'assists team2',
       'on target attempts team1', 'on target attempts team2',
       'off target attempts team1', 'off target attempts team2',
       'attempts inside the penalty area team1',
       'attempts inside the penalty area  team2',
       'attempts outside the penalty area  team1',
       'attempts outside the penalty area  team2', 'left channel team1',
       'left channel team2', 'left inside channel team1',
       'left inside channel team2', 'central channel team1',
       'central channel team2', 'right in

In [3]:
df.columns

Index(['team1', 'team2', 'possession team1', 'possession team2',
       'possession in contest', 'number of goals team1',
       'number of goals team2', 'date', 'hour', 'category',
       'total attempts team1', 'total attempts team2', 'conceded team1',
       'conceded team2', 'goal inside the penalty area team1',
       'goal inside the penalty area team2',
       'goal outside the penalty area team1',
       'goal outside the penalty area team2', 'assists team1', 'assists team2',
       'on target attempts team1', 'on target attempts team2',
       'off target attempts team1', 'off target attempts team2',
       'attempts inside the penalty area team1',
       'attempts inside the penalty area  team2',
       'attempts outside the penalty area  team1',
       'attempts outside the penalty area  team2', 'left channel team1',
       'left channel team2', 'left inside channel team1',
       'left inside channel team2', 'central channel team1',
       'central channel team2', 'right in

In [3]:
def row_to_text(row):
    # Adjust column names to your actual CSV headers
    return (
       
        f"FIFA World Cup 2022 Match {row['team1']} vs {row['team2']}. "
        f"Score: {row['number of goals team1']} - {row['number of goals team2']}. "
        f"Date: {row['date']}. "
        f"Category:{row['category']} ,"
        f"Possesion of {row['team1']} is {row['possession team1']} and {row['team2']} is {row['possession team2']} "
        f"{row['team1']} had {row['on target attempts team1']} On Target Attempts and {row['team2']} had  {row['on target attempts team2']} attempts"
         f"{row['team1']} had {row['assists team1']} assists and {row['team2']} had {row['assists team2']} assists"
    )

docs = []
for _, row in df.iterrows():
    text = row_to_text(row)
    metadata = {
        "team1": row["team1"],
        "team2": row["team2"],
        "category":row['category']
    }
    docs.append(Document(page_content=text, metadata=metadata))

len(docs), docs[0]


(64,
 Document(metadata={'team1': 'QATAR', 'team2': 'ECUADOR', 'category': 'Group A'}, page_content='FIFA World Cup 2022 Match QATAR vs ECUADOR. Score: 0 - 2. Date: 20 NOV 2022. Category:Group A ,Possesion of QATAR is 42% and ECUADOR is 50% QATAR had 0 On Target Attempts and ECUADOR had  3 attemptsQATAR had 0 assists and ECUADOR had 1 assists'))

In [4]:
print(docs)

[Document(metadata={'team1': 'QATAR', 'team2': 'ECUADOR', 'category': 'Group A'}, page_content='FIFA World Cup 2022 Match QATAR vs ECUADOR. Score: 0 - 2. Date: 20 NOV 2022. Category:Group A ,Possesion of QATAR is 42% and ECUADOR is 50% QATAR had 0 On Target Attempts and ECUADOR had  3 attemptsQATAR had 0 assists and ECUADOR had 1 assists'), Document(metadata={'team1': 'ENGLAND', 'team2': 'IRAN', 'category': 'Group B'}, page_content='FIFA World Cup 2022 Match ENGLAND vs IRAN. Score: 6 - 2. Date: 21 NOV 2022. Category:Group B ,Possesion of ENGLAND is 72% and IRAN is 19% ENGLAND had 7 On Target Attempts and IRAN had  3 attemptsENGLAND had 6 assists and IRAN had 1 assists'), Document(metadata={'team1': 'SENEGAL', 'team2': 'NETHERLANDS', 'category': 'Group A'}, page_content='FIFA World Cup 2022 Match SENEGAL vs NETHERLANDS. Score: 0 - 2. Date: 21 NOV 2022. Category:Group A ,Possesion of SENEGAL is 44% and NETHERLANDS is 45% SENEGAL had 3 On Target Attempts and NETHERLANDS had  3 attemptsSEN

INTIATING THE EMBEDDING MODEL

In [5]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3",
)



In [6]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(docs, embedding=embeddings)


Now that the vector embeddings are completed we will now be doing a test for similarity search so that we can see what is actually going on here now using similiratiy_score_threshhold

similarity = “just give me closest stuff”

similarity_score_threshold = “closest stuff but drop trash”

mmr = “closest but also diverse (good for longer answers)”

In [7]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 10,                 # consider up to 10
        "score_threshold": 0.9   # filter out low similarity
    }
)



In [ ]:
retriever_s = vector_store.as_retriever(
    search_type="similarity",               # or "similarity_score_threshold", "mmr"
    search_kwargs={"k": 5}                  # top 5 matches
)


In [9]:
prompt = "Tell me about the match between Portugal and Morocco"  # ⬅️ Add this

results = retriever_s.invoke(prompt)   # ⬅️ this replaces get_relevant_documents

for doc in results:
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 40)

FIFA World Cup 2022 Match MOROCCO vs PORTUGAL. Score: 1 - 0. Date: 10 DEC 2022. Category:Quarter-final ,Possesion of MOROCCO is 22% and PORTUGAL is 65% MOROCCO had 3 On Target Attempts and PORTUGAL had  3 attemptsMOROCCO had 1 assists and PORTUGAL had 0 assists
{'team1': 'MOROCCO', 'team2': 'PORTUGAL', 'category': 'Quarter-final'}
----------------------------------------
FIFA World Cup 2022 Match MOROCCO vs SPAIN. Score: 0 - 0. Date: 06 DEC 2022. Category:Round of 16 ,Possesion of MOROCCO is 22% and SPAIN is 68% MOROCCO had 3 On Target Attempts and SPAIN had  1 attemptsMOROCCO had 0 assists and SPAIN had 0 assists
{'team1': 'MOROCCO', 'team2': 'SPAIN', 'category': 'Round of 16'}
----------------------------------------
FIFA World Cup 2022 Match MOROCCO vs CROATIA. Score: 0 - 0. Date: 23 NOV 2022. Category:Group F ,Possesion of MOROCCO is 32% and CROATIA is 57% MOROCCO had 2 On Target Attempts and CROATIA had  2 attemptsMOROCCO had 0 assists and CROATIA had 0 assists
{'team1': 'MOROCCO'

Now Setting Up The Actualy LLM and then seeing what can be done

In [10]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from langchain.tools import tool,ToolRuntime

In [11]:
llm = ChatOllama(
    model="llama3.2:3b",  # or any local ollama model name you’ve pulled
    temperature=0.2,
)

checkpointer = InMemorySaver()

In [15]:

from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(retriever_s,name='retriever_data',description='Usign this tool we will retrive the data from the vector store')



In [17]:
@tool
def retrieve_data(query: str) -> str:
    """Retrieve World Cup match data from the vector store"""
    results = retriever_s.invoke(query)
    return "\n\n".join([doc.page_content for doc in results])

checkpointer = InMemorySaver()

agent = create_agent(
    model=llm,
    tools=[retrieve_data],
    checkpointer=checkpointer,
    system_prompt="""You are a World Cup football data assistant.
First call retrieve_data to retrieve context.
Use ONLY the data from the retrieve_data tool to answer the question.
If the data does not contain the answer, say so clearly."""
)


In [28]:
config={"configurable": {"thread_id": "9"}}

result=agent.invoke({
    "messages":[{"role":"user","content":"Hey I am Ananda Prem How are you doing"}]
},config)

result=agent.invoke({
    "messages":[{"role":"user","content":"What happened in the match between  Portugal and Morroco in the quarter finals"}]
},config)

result_2=agent.invoke({
    "messages":[{"role":"user","content":"What would you say about the Possession Stats Of Both The Teams can you please list it out for me also"}]
},config)

result_3=agent.invoke({
    "messages":[{"role":"user","content":"What was the reason that Portugal lost can you please tell i was really looking forward to Portugal Winning"}]
},config)


In [31]:
print("Result 1:")
print(result["messages"][-1].content)

print("\nResult 2:")
print(result_2["messages"][-1].content)

print("\nResult 3:")
print(result_3["messages"][-1].content)

Result 1:
The match between Portugal and Morocco in the quarter-finals of the FIFA World Cup 2022 was a closely contested game. The final score was 1-0, with Morocco emerging as the winner.

Morocco had significantly less possession than Portugal, with only 22% of the ball compared to Portugal's 65%. Despite this, Morocco still managed to create scoring opportunities and keep the game close.

The on-target attempts and assists stats suggest that both teams were evenly matched in terms of creating chances. However, Morocco's defensive solidity and ability to limit Portugal's scoring opportunities ultimately led to their victory.

It's worth noting that this match was a significant upset, as Morocco was considered an underdog going into the game. Their victory will be remembered as one of the biggest surprises of the tournament.

As for what happened in the match, Morocco scored the only goal of the game in the 80th minute, securing their spot in the semi-finals. The goal was scored by [

In [32]:
print("Result 3 (Full):")
print(result_3["messages"][-1].content)


Result 3 (Full):
Portugal lost the match against Morocco in the quarter-finals of the 2022 FIFA World Cup due to a combination of factors.

One major reason was Morocco's defensive solidity, which limited Portugal's scoring opportunities. Despite having more possession, Portugal was unable to create clear-cut chances and was often caught off guard by Morocco's counter-attacks.

Another factor was Morocco's ability to capitalize on their own chances. They had three on-target attempts, all of which were successful, while Portugal had only three on-target attempts, none of which were successful.

Additionally, Portugal's lack of creativity in the midfield and attack was a significant issue. They were unable to create sustained pressure on the Moroccan goal, and their attacks often broke down due to poor decision-making or lack of quality in front of goal.

Overall, it was a disappointing loss for Portugal, but they can take comfort in knowing that they gave it their all and pushed Morocco

In [30]:
result["messages"][-1].pretty_print()
result_2["messages"][-1].pretty_print()
result_3["messages"][-1].pretty_print()


================================== Ai Message ==================================

The match between Portugal and Morocco in the quarter-finals of the FIFA World Cup 2022 was a closely contested game. The final score was 1-0, with Morocco emerging as the winner.

Morocco had significantly less possession than Portugal, with only 22% of the ball compared to Portugal's 65%. Despite this, Morocco still managed to create scoring opportunities and keep the game close.

The on-target attempts and assists stats suggest that both teams were evenly matched in terms of creating chances. However, Morocco's defensive solidity and ability to limit Portugal's scoring opportunities ultimately led to their victory.

It's worth noting that this match was a significant upset, as Morocco was considered an underdog going into the game. Their victory will be remembered as one of the biggest surprises of the tournament.

As for what happened in the match, Morocco scored the only goal of the game in the 80th 

In [ ]:
config={"configurable": {"thread_id": "1"}}

agent.invoke({"messages": "hi, my name is Nathan"}, config)
resp=agent.invoke({"messages": "Tell me about Portugals Match Against Morroco"}, config)
resp=agent.invoke({"messages": "What Would You tell About the possession did portugal deserve to win based off the possesion they maintained throught the match"}, config)
resp["messages"][-1].pretty_print()
resp=agent.invoke({"messages":"What is my name again ?"},config)
resp["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have any specific information about a recent match between Portugal and Morocco. However, I can suggest some general information about the teams.

Portugal is a European country that has a rich football history and has produced many talented players over the years. They have won several international titles, including the European Championship and the FIFA World Cup.

Morocco is also a European country with a strong football tradition. They have participated in several international tournaments, including the FIFA World Cup and the African Cup of Nations.

If you're interested in learning more about their matches against each other, I can suggest some general information about their head-to-head record.

Would you like to know more about Portugal's or Morocco's football history?
================================== Ai Message ==================================

I don't have any specific information 

Creating a Tool to Retrive The Data From The Vector Store

In [ ]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage
system_msg = SystemMessage("""You are a World Cup football data assistant.

Use ONLY the context below to answer the question.
If the context does not contain the answer, say so clearly.

Context:
{context}

Question:
{question}""")
